<a href="https://colab.research.google.com/github/pawo161/LSTM/blob/master/LSTM_music_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install music21
!pip3 install keras

In [0]:
import os
import time

from zipfile import ZipFile
import glob
import pickle
import numpy as np
import sys
import keras
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import CuDNNLSTM, LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import TensorBoard
from shutil import copyfile
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from music21 import converter, corpus, instrument, midi, note, chord, pitch
from music21 import *

Using TensorFlow backend.


In [0]:
"""
1. dataset Maestro od Magenta
https://magenta.tensorflow.org/datasets/maestro
midi only (53 mb)
2. mount drive 
3. unpack and preprocess
glob.glob('/content/drive/My Drive/midi/*/*.midi')
"""

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
zipfile = "/content/drive/My Drive/maestro-v2.0.0-midi.zip"
with ZipFile(zipfile, 'r') as zipObj:
   zipObj.extractall('/content/drive/My Drive/')

In [0]:
notes=[]

In [0]:
#definiowanie funkcji do czytania MIDI
def read_midi(file):
    
    print("ładuję midi:",file)
    #parsing a midi file
    midi = converter.parse(file)
    
    notes_to_parse = None
    notes_to_parse = midi.recurse() 
    #gdyby pliki mialy podzial na instrumenty (moje mają tylko jeden):
#     parts = instrument.partitionByInstrument(midi)
    
    #parsing a midi file
#     midi = converter.parse(file)
    
    # s2 = instrument.partitionByInstrument(midi)

    #wykrywanie czy jest akord czy pojedynczy dźwięk
#     for element in notes_to_parse:
#     if parts: # file has instrument parts
#          notes_to_parse = parts.parts[0].recurse()
#     else: # file has notes in a flat structure
#          notes_to_parse = midi.flat.notes
    for element in enumerate(notes_to_parse):   
        #nuta
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
#             print("nuta:", str(element.pitch))
        #akord
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n)for n in element.normalOrder))
#             print("akord:", '.'.join(str(n) for n in element.normalOrder))
        elif isinstance(element, note.Rest):
            notes.append('#'+str(element.seconds))
#             print("pauza:", '#'+str(element.seconds))
        
    with open('jazz', 'wb') as filepath:
        pickle.dump(notes, filepath)       
    return notes

In [0]:
#funkcja odpalająca zbieranie danych midi do listy
for f in glob.glob('/content/drive/My Drive/maestro-v2.0.0/2011/*.midi'):
    read_midi(f)
    

In [0]:
# test pojedynczego pliku w zapisie music21 np. E3 to nuta E w 3. oktawie, E-3 to Eflat w trzeciej oktawie
# akordy zapisane są jako dzwięki zapisie midi oddzielone kropkami
#pauza to wartość z '#' na początku np. 0.125 to ósemka
read_midi('/content/drive/My Drive/maestro-v2.0.0/2017/MIDI-Unprocessed_041_PIANO041_MID--AUDIO-split_07-06-17_Piano-e_1-01_wav--1.midi')

In [0]:
#jeśli plik pickle jest już utworzony
with open('/content/drive/My Drive/newest_dump_2011', 'rb') as filepath:
        notes = pickle.load(filepath)

In [0]:
#liczba wszystkich możliwych wartości
# notes_ = [element for note_ in notes_array for element in note_]
# print(notes_[:10])
# #unique notes
# n_vocab = len(set(notes_))
n_vocab = len(set(notes))
n_vocab

1458

In [0]:
#przygotowuję dane do wejścia LSTM

#długość sekwencji wejściowej, której następnik sieć ma przewidzieć
sequence_length = 100
# zbieram wszystkie nuty, posortowane (notacja midi 1 - 128)
pitchnames = sorted(set(item for item in notes))

# tokenizuję, przyporządkowując każdej nucie liczbę
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []

# sekwencje wejściowe i odpowiadające im wyjścia
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length] # ma długość sekwencji (wybrałem 40)
    # print("to na wejście sieci:", sequence_in)
    sequence_out = notes[i + sequence_length]  # długość 1, nuta do przewidzenia przez sieć jako kolejna po sekwencji na wejściu
    # print("to na wyjście sieci:", sequence_out)

    # # Przyporządkowujemy sekwencji wejściowej liczby
    network_input.append([note_to_int[char] for char in sequence_in])
    # print("to na wejście sieci jako int:", sequence_in)
    # # Przyporządkowujemy sekwencji wyjściowej liczby
    network_output.append(note_to_int[sequence_out])
    # print("to na wyjście sieci jako int:", sequence_out)

n_patterns = len(network_input)
# dostosowanie formatu wejściowego odpowiedniego dla warstw LSTM
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# normalizacja wejścia
network_input = network_input / n_vocab
# n_vocab = np.max(network_input)
# print(n_vocab)
#Converts a class vector (integers) to binary class matrix.
#for use with categorical_crossentropy.
network_output = np_utils.to_categorical(network_output)
print(network_output)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:
#używam keras, który działa na tensorflow 2+,
#jeśli użyję odpowienich parametrów w warstwach lstm domyślnie użyte będzie GPU
def create_network(network_input, n_vocab):
      model = Sequential()

      model.add(LSTM(128, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
      model.add(Dropout(0.3))
      model.add(BatchNormalization())

      model.add(LSTM(128, return_sequences=True))
      model.add(Dropout(0.3))
      model.add(BatchNormalization())

      model.add(LSTM(256))
      model.add(Dropout(0.3))
      model.add(BatchNormalization())

      model.add(Dense(128, activation="relu"))
      model.add(Dropout(0.3))
      model.add(BatchNormalization())

      model.add(Dense(n_vocab))
      model.add(Activation('softmax'))
      model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=["accuracy"])
      return model

In [0]:
#testuję kilka sieci, różne prarametry
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(256))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=["accuracy"])
    return model

In [0]:
#podsumowanie sieci
model = create_network(network_input, n_vocab)
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 40, 256)           264192    
_________________________________________________________________
dropout_18 (Dropout)         (None, 40, 256)           0         
_________________________________________________________________
lstm_17 (LSTM)               (None, 40, 512)           1574912   
_________________________________________________________________
dropout_19 (Dropout)         (None, 40, 512)           0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 256)               787456    
_________________________________________________________________
dense_10 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_20 (Dropout)         (None, 256)              

In [0]:
#rozpoczęcie treningu

filepath = "train_n_vocab{epoch:02d}-{loss:.4f}.h5"

checkpoint = ModelCheckpoint(filepath, verbose=0,
                             monitor='loss',save_best_only=True, mode='min')

callbacks_list = [checkpoint]
model.fit(network_input, network_output, epochs=40, batch_size=256, 
          callbacks=callbacks_list)


In [0]:
# ewaluacja wykonanego treningu 
print("\nModel evaluation at the end of training")
train_acc = model.evaluate(network_input, network_output, verbose=1)
print(model.metrics_names)
print(train_acc)


Model evaluation at the end of training
  3936/479421 [..............................] - ETA: 21:51

In [0]:
# Sekcja generowania wyników :
from keras.models import load_model

model.load_weights("/content/train_n_vocab16-4.4654.h5")


In [0]:
from tqdm import trange
#mała paczka do progress baru
import random
print('generating prediction stream...')
 #wybieram losową sekwencję z wejścia jak punkt startowy do generowania (predykcji)
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start].tolist()
prediction_output = []

generating prediction stream...


In [0]:
#wybrana ilość nut do wygenerowania (200)
for note_index in trange(200):
     prediction_input = np.reshape(pattern, (1, len(pattern), 1))
     prediction = model.predict(prediction_input, verbose=0)
     index = np.argmax(prediction)
     result = int_to_note[index]
     prediction_output.append(result)
     pattern.append([index/49])
     pattern = pattern[1:len(pattern)]

100%|██████████| 60/60 [00:01<00:00, 58.36it/s]


In [0]:
offset = 0
output_notes = []

# kreowanie obiektów music21 z wygenerowanej predykcji
print('creating notes and chords')
for pattern in prediction_output:
    
    # pattern to akord
    if (('.' in pattern) or pattern.isdigit()) and pattern[0]!='#':
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
        
    # pattern to nuta
    elif pattern[0]!='#':
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        
    # pattern to pauza
    else:
        new_note = note.Rest()
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        new_note.quarterLength = float(pattern[1:])
        output_notes.append(new_note)
    # odstęp, aby nuty nie nakładały się na siebie
    offset += 0.5

creating notes and chords


In [0]:
#zmieniamy wygenerowane dane na nuty i tworzymy plik midi z tych nut
midi_stream = stream.Stream(output_notes)
print(output_notes)
midi_stream.write('midi', fp="fragment13.mid")

[<music21.note.Note E>, <music21.note.Note E>, <music21.note.Note D>, <music21.note.Note D>, <music21.note.Note D>, <music21.note.Note D>, <music21.note.Note D>, <music21.note.Note D>, <music21.note.Note D>, <music21.note.Note E->, <music21.note.Note E->, <music21.note.Note E->, <music21.note.Note E->, <music21.note.Note D>, <music21.note.Note E->, <music21.note.Note E->, <music21.note.Note E->, <music21.note.Note E->, <music21.note.Note E->, <music21.note.Note E->, <music21.note.Note E->, <music21.note.Note D>, <music21.note.Note E->, <music21.note.Note F#>, <music21.note.Note F#>, <music21.note.Note F#>, <music21.note.Note F#>, <music21.note.Note F#>, <music21.note.Note F#>, <music21.note.Note F#>, <music21.note.Note F#>, <music21.note.Note F#>, <music21.note.Note F#>, <music21.note.Note F#>, <music21.note.Note F#>, <music21.note.Note F#>, <music21.note.Note F#>, <music21.note.Note F#>, <music21.note.Note F#>, <music21.note.Note F#>, <music21.note.Note E->, <music21.note.Note E->, <m

'fragment13.mid'